# Climate-Related Development Finance (CRDF) Analysis

This notebook analyzes climate finance data from 2000-2023 with the following visualizations:

1. Climate Finance Over Time (Trend)
2. Top Recipient Countries (Distribution)
3. Finance by Region (Stacked Bar)
4. Adaptation vs Mitigation Balance
5. Distribution by Income Group (Equity)
6. Sector Allocation of Climate Finance
7. Provider → Recipient Network (Flows)
8. Project-Level Impact Proxy
9. Channel of Delivery Analysis

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Load the CRDF dataset
file_path = 'CRDF-RP-all years-2000-2023.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Explore the data structure
print("Data info:")
print(df.info())
print("\n\nMissing values:")
print(df.isnull().sum())
print("\n\nSample data:")
df.head(10)

## 1. Climate Finance Over Time (Trend)

In [ ]:
# Clean and prepare the data
# Convert Year to numeric if needed
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Convert climate finance column to numeric
climate_finance_col = 'Climate-related development finance - Commitment - 2023 USD thousand'
df[climate_finance_col] = pd.to_numeric(df[climate_finance_col], errors='coerce')

# Aggregate by year
finance_by_year = df.groupby('Year')[climate_finance_col].sum().sort_index()

# Create line chart
plt.figure(figsize=(14, 6))
plt.plot(finance_by_year.index, finance_by_year.values / 1e6, marker='o', linewidth=2, markersize=8, color='darkgreen')
plt.title('Climate Finance Over Time', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nClimate Finance by Year (2023 USD, billions):")
print((finance_by_year / 1e6).round(2))

In [ ]:
# Optional: Breakdown by Region
if 'Recipient Region' in df.columns:
    finance_by_year_region = df.groupby(['Year', 'Recipient Region'])[climate_finance_col].sum().reset_index()
    
    plt.figure(figsize=(16, 8))
    for region in finance_by_year_region['Recipient Region'].unique():
        region_data = finance_by_year_region[finance_by_year_region['Recipient Region'] == region]
        plt.plot(region_data['Year'], region_data[climate_finance_col] / 1e6, 
                marker='o', linewidth=2, label=region, markersize=6)
    
    plt.title('Climate Finance Over Time by Region', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Year', fontsize=12, fontweight='bold')
    plt.ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Optional: Breakdown by Income Group
if 'Recipient Income Group (OECD Classification)' in df.columns:
    finance_by_year_income = df.groupby(['Year', 'Recipient Income Group (OECD Classification)'])[climate_finance_col].sum().reset_index()
    
    plt.figure(figsize=(16, 8))
    for income in finance_by_year_income['Recipient Income Group (OECD Classification)'].unique():
        income_data = finance_by_year_income[finance_by_year_income['Recipient Income Group (OECD Classification)'] == income]
        plt.plot(income_data['Year'], income_data[climate_finance_col] / 1e6, 
                marker='o', linewidth=2, label=income, markersize=6)
    
    plt.title('Climate Finance Over Time by Income Group', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Year', fontsize=12, fontweight='bold')
    plt.ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 2. Top Recipient Countries (Distribution)

In [ ]:
# Aggregate by recipient country
if 'Recipient' in df.columns:
    finance_by_recipient = df.groupby('Recipient')[climate_finance_col].sum().sort_values(ascending=False)
    
    # Get top 15 countries
    top_recipients = finance_by_recipient.head(15)
    
    # Create bar chart
    plt.figure(figsize=(14, 8))
    top_recipients.plot(kind='barh', color='steelblue', edgecolor='black')
    plt.title('Top 15 Recipient Countries - Climate Finance', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Climate Finance (2023 USD, thousands)', fontsize=12, fontweight='bold')
    plt.ylabel('Recipient Country', fontsize=12, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 15 Recipient Countries:")
    print((top_recipients / 1e6).round(2))

## 3. Finance by Region (Stacked Bar)

In [ ]:
# Prepare data for stacked bar chart
adaptation_col = 'Adaptation-related development finance (includes overlap) - Commitment - 2023 USD thousand'
mitigation_col = 'Mitigation-related development finance (includes overlap) - Commitment - 2023 USD thousand'
overlap_col = 'Overlap - Commitment - 2023 USD thousand'

# Convert to numeric
df[adaptation_col] = pd.to_numeric(df[adaptation_col], errors='coerce')
df[mitigation_col] = pd.to_numeric(df[mitigation_col], errors='coerce')
df[overlap_col] = pd.to_numeric(df[overlap_col], errors='coerce')

if 'Recipient Region' in df.columns:
    # Aggregate by region
    region_finance = df.groupby('Recipient Region').agg({
        adaptation_col: 'sum',
        mitigation_col: 'sum',
        overlap_col: 'sum'
    })
    region_finance['Total'] = region_finance[adaptation_col] + region_finance[mitigation_col] + region_finance[overlap_col]
    region_finance = region_finance.sort_values('Total', ascending=False)
    
    # Create stacked bar chart
    fig, ax = plt.subplots(figsize=(14, 8))
    
    x = range(len(region_finance))
    width = 0.6
    
    # Plot stacked bars
    p1 = ax.bar(x, region_finance[adaptation_col] / 1e6, width, label='Adaptation', color='#2E86AB', alpha=0.8)
    p2 = ax.bar(x, region_finance[mitigation_col] / 1e6, width, 
                bottom=region_finance[adaptation_col] / 1e6, label='Mitigation', color='#A23B72', alpha=0.8)
    p3 = ax.bar(x, region_finance[overlap_col] / 1e6, width,
                bottom=(region_finance[adaptation_col] + region_finance[mitigation_col]) / 1e6, 
                label='Overlap', color='#F18F01', alpha=0.8)
    
    ax.set_xlabel('Recipient Region', fontsize=12, fontweight='bold')
    ax.set_ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
    ax.set_title('Climate Finance by Region (Adaptation/Mitigation/Overlap)', fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(region_finance.index, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nFinance by Region (billions USD):")
    print((region_finance[[adaptation_col, mitigation_col, overlap_col]] / 1e6).round(2))

## 4. Adaptation vs Mitigation Balance

In [ ]:
# Aggregate by year
yearly_balance = df.groupby('Year').agg({
    adaptation_col: 'sum',
    mitigation_col: 'sum',
    overlap_col: 'sum'
}).sort_index()

# Create stacked area chart
plt.figure(figsize=(14, 8))
plt.stackplot(yearly_balance.index, 
              yearly_balance[adaptation_col] / 1e6,
              yearly_balance[mitigation_col] / 1e6,
              yearly_balance[overlap_col] / 1e6,
              labels=['Adaptation', 'Mitigation', 'Overlap'],
              colors=['#2E86AB', '#A23B72', '#F18F01'],
              alpha=0.7)

plt.title('Adaptation vs Mitigation Balance Over Time', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Also create grouped bar chart
fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(yearly_balance))
width = 0.25

ax.bar(x - width, yearly_balance[adaptation_col] / 1e6, width, label='Adaptation', color='#2E86AB', alpha=0.8)
ax.bar(x, yearly_balance[mitigation_col] / 1e6, width, label='Mitigation', color='#A23B72', alpha=0.8)
ax.bar(x + width, yearly_balance[overlap_col] / 1e6, width, label='Overlap', color='#F18F01', alpha=0.8)

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
ax.set_title('Adaptation vs Mitigation Balance (Grouped Bars)', fontsize=16, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(yearly_balance.index, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Distribution by Income Group (Equity)

In [ ]:
if 'Recipient Income Group (OECD Classification)' in df.columns:
    # Box plot
    income_data = df[df[climate_finance_col].notna() & df['Recipient Income Group (OECD Classification)'].notna()]
    
    plt.figure(figsize=(12, 8))
    income_groups = income_data['Recipient Income Group (OECD Classification)'].unique()
    data_to_plot = [income_data[income_data['Recipient Income Group (OECD Classification)'] == group][climate_finance_col].dropna() / 1e6 
                    for group in income_groups]
    
    bp = plt.boxplot(data_to_plot, labels=income_groups, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
        patch.set_alpha(0.7)
    
    plt.title('Climate Finance Distribution by Income Group (Box Plot)', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Recipient Income Group', fontsize=12, fontweight='bold')
    plt.ylabel('Climate Finance (2023 USD, millions)', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Bar chart (aggregated totals)
    income_totals = df.groupby('Recipient Income Group (OECD Classification)')[climate_finance_col].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 8))
    income_totals.plot(kind='bar', color='coral', edgecolor='black', alpha=0.8)
    plt.title('Total Climate Finance by Income Group', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Recipient Income Group', fontsize=12, fontweight='bold')
    plt.ylabel('Climate Finance (2023 USD, thousands)', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nTotal Finance by Income Group:")
    print((income_totals / 1e6).round(2))

## 6. Sector Allocation of Climate Finance

In [ ]:
if 'Sector' in df.columns:
    # Aggregate by sector
    sector_finance = df.groupby('Sector').agg({
        adaptation_col: 'sum',
        mitigation_col: 'sum'
    })
    sector_finance['Total'] = sector_finance[adaptation_col] + sector_finance[mitigation_col]
    sector_finance = sector_finance.sort_values('Total', ascending=False).head(10)
    
    # Create stacked bar chart
    fig, ax = plt.subplots(figsize=(14, 8))
    
    x = range(len(sector_finance))
    width = 0.6
    
    p1 = ax.bar(x, sector_finance[adaptation_col] / 1e6, width, label='Adaptation', color='#2E86AB', alpha=0.8)
    p2 = ax.bar(x, sector_finance[mitigation_col] / 1e6, width, 
                bottom=sector_finance[adaptation_col] / 1e6, label='Mitigation', color='#A23B72', alpha=0.8)
    
    ax.set_xlabel('Sector', fontsize=12, fontweight='bold')
    ax.set_ylabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
    ax.set_title('Top 10 Sectors - Climate Finance Allocation', fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(sector_finance.index, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Sectors (billions USD):")
    print((sector_finance[[adaptation_col, mitigation_col]] / 1e6).round(2))

## 7. Provider → Recipient Network (Flows)

In [ ]:
# For Sankey diagram, we'll use plotly if available, otherwise create a simplified version
try:
    import plotly.graph_objects as go
    
    # Get top providers and recipients
    if 'Provider (detailed)' in df.columns and 'Recipient' in df.columns:
        # Aggregate flows
        flows = df.groupby(['Provider (detailed)', 'Recipient'])[climate_finance_col].sum().reset_index()
        flows = flows[flows[climate_finance_col] > 0].sort_values(climate_finance_col, ascending=False).head(50)
        
        # Create node lists
        providers = flows['Provider (detailed)'].unique()
        recipients = flows['Recipient'].unique()
        
        all_nodes = list(providers) + list(recipients)
        node_indices = {node: i for i, node in enumerate(all_nodes)}
        
        # Create source, target, and value lists
        source = [node_indices[p] for p in flows['Provider (detailed)']]
        target = [node_indices[r] for r in flows['Recipient']]
        value = flows[climate_finance_col].tolist()
        
        # Create Sankey diagram
        fig = go.Figure(data=[go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=all_nodes
            ),
            link=dict(
                source=source,
                target=target,
                value=value
            )
        )])
        
        fig.update_layout(title_text="Provider → Recipient Climate Finance Flows", font_size=12, height=800)
        fig.show()
        
except ImportError:
    print("Plotly not available. Creating simplified flow visualization...")
    
    # Simplified version: Top provider-recipient pairs
    if 'Provider (detailed)' in df.columns and 'Recipient' in df.columns:
        flows = df.groupby(['Provider (detailed)', 'Recipient'])[climate_finance_col].sum().reset_index()
        top_flows = flows.nlargest(20, climate_finance_col)
        
        plt.figure(figsize=(14, 10))
        y_pos = np.arange(len(top_flows))
        plt.barh(y_pos, top_flows[climate_finance_col] / 1e6)
        plt.yticks(y_pos, [f"{p} → {r}" for p, r in zip(top_flows['Provider (detailed)'], top_flows['Recipient'])])
        plt.xlabel('Climate Finance (2023 USD, billions)', fontsize=12, fontweight='bold')
        plt.title('Top 20 Provider → Recipient Flows', fontsize=16, fontweight='bold', pad=20)
        plt.gca().invert_yaxis()
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()

## 8. Project-Level Impact Proxy

In [ ]:
# A. Funding vs Climate Objective
if 'Climate objective (applies to Rio-marked data only or climate component)' in df.columns:
    climate_obj_data = df[df[climate_finance_col].notna() & df['Climate objective (applies to Rio-marked data only or climate component)'].notna()]
    
    plt.figure(figsize=(12, 8))
    objectives = climate_obj_data['Climate objective (applies to Rio-marked data only or climate component)'].unique()
    data_to_plot = [climate_obj_data[climate_obj_data['Climate objective (applies to Rio-marked data only or climate component)'] == obj][climate_finance_col].dropna() / 1e6 
                    for obj in objectives]
    
    bp = plt.boxplot(data_to_plot, labels=objectives, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor('lightgreen')
        patch.set_alpha(0.7)
    
    plt.title('Funding vs Climate Objective', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Climate Objective', fontsize=12, fontweight='bold')
    plt.ylabel('Climate Finance (2023 USD, millions)', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# B. Funding vs Adaptation/Mitigation Marker
adaptation_obj_col = 'Adaptation objective (applies to Rio-marked data only)'
mitigation_obj_col = 'Mitigation objective (applies to Rio-marked data only)'

if adaptation_obj_col in df.columns and mitigation_obj_col in df.columns:
    # Create a combined analysis
    obj_data = df[df[climate_finance_col].notna()].copy()
    
    # Aggregate by adaptation and mitigation objectives
    adaptation_totals = obj_data.groupby(adaptation_obj_col)[climate_finance_col].sum()
    mitigation_totals = obj_data.groupby(mitigation_obj_col)[climate_finance_col].sum()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Adaptation objectives
    adaptation_totals.plot(kind='bar', ax=ax1, color='#2E86AB', alpha=0.8, edgecolor='black')
    ax1.set_title('Funding by Adaptation Objective', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Adaptation Objective', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Climate Finance (2023 USD, thousands)', fontsize=11, fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Mitigation objectives
    mitigation_totals.plot(kind='bar', ax=ax2, color='#A23B72', alpha=0.8, edgecolor='black')
    ax2.set_title('Funding by Mitigation Objective', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Mitigation Objective', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Climate Finance (2023 USD, thousands)', fontsize=11, fontweight='bold')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 9. Channel of Delivery Analysis

In [ ]:
if 'Channel of Delivery' in df.columns:
    channel_finance = df.groupby('Channel of Delivery')[climate_finance_col].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(14, 8))
    channel_finance.plot(kind='bar', color='purple', edgecolor='black', alpha=0.8)
    plt.title('Climate Finance by Channel of Delivery', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Channel of Delivery', fontsize=12, fontweight='bold')
    plt.ylabel('Climate Finance (2023 USD, thousands)', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nFinance by Channel of Delivery:")
    print((channel_finance / 1e6).round(2))